This notebooks is modified based on the AESPM tutorial linked here: https://github.com/RichardLiuCoding/aespm/tree/main/aespm/notebooks

In this notebook, I have changed all the AESPM functions to their equivalent ones in the RePySPM library.

Here many approaching and motor status checking functions are missing in the library and I have put some placeholders in the last cell.

# Import Packages

In [1]:
from igor2 import binarywave as bw

import time
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import shutil

# import aespm as ae
from afm_controller import AFMController

In [2]:
# Step 1: Initialize the AFM Controller
afm = AFMController()

# Step 2: Connect to the AFM system
afm.connect()

# AC mode exploration

In [ ]:
# Set the scan parameters for the exploration
# Please skip this step if you prefer setting the initial parameters manually 
print("\n--- Setting Scan Parameters ---")
afm.scan_parameters.set_scan_parameters(
    width=10e-6,  # 10 µm
    height=10e-6, # 10 µm
    rotation=0,
    speed=1e-6    # 1 µm/s
)
print("Scan parameters set.")

In [ ]:
# Define the grid-size
# Assume this is a 1D exploration along the x-axis with a total distance L
displacement = [L, 0]
num_points = 20

In [ ]:
# Initial sample height
# This is the sample surface height across the exploration distance, D
# If the values here are incorrect, the probe will crash into the sample surface or it takes forever to approach!
sample_height = [1.8e-3] * num_points

offset = 150e-6 # the pre-engage height is 150 um from the sample surface

for i in range(num_points):
    print("Working on Location: {}/{}".format(i+1, num_points), end='\r')

    # Set the new sample height
    afm.motors.set_sample_height(sample_height[i])
    # Move the probe to the pre-engage height
    afm.z_control.moveZ_stage_distance(sample_height[i] + offset)
    # Start the approaching process
    afm.z_control.start_approach()
    
    # Wait until the approaching to finish
    max_wait = 5 * 60 # let's wait no longer than 5 min
    N = 0
    
    while afm.z_control.is_approaching() and N <= int(max_wait / 10):
        time.sleep(10)
        N += 1

    # Start the topography scan and wait until it's done
    afm.scan_control.scan_up()

    max_wait = 5 * 60 # let's wait no longer than 5 min
    N = 0
    
    while afm.scan_control.is_scanning() and N <= int(max_wait / 10):
        time.sleep(10)
        N += 1

    # Next we withdraw the probe and move to the next location to measure
    afm.z_control.retract()
    afm.motors.moveXstage(L/num_points)
    # Wait until the moving is done